# Find the Most Frequent Words with Mismatches in a String

[ba1i](https://rosalind.info/problems/ba1i/)

We defined a mismatch in “Compute the Hamming Distance Between Two Strings”. We now generalize “Find the Most Frequent Words in a String” to incorporate mismatches as well.

Given strings Text and Pattern as well as an integer d, we define Countd(Text, Pattern) as the total number of occurrences of Pattern in Text with at most d mismatches. For example, Count1(AACAAGCTGATAAACATTTAAAGAG, AAAAA) = 4 because AAAAA appears four times in this string with at most one mismatch: AACAA, ATAAA, AAACA, and AAAGA. Note that two of these occurrences overlap.

A most frequent k-mer with up to d mismatches in Text is simply a string Pattern maximizing Countd(Text, Pattern) among all k-mers. Note that Pattern does not need to actually appear as a substring of Text; for example, AAAAA is the most frequent 5-mer with 1 mismatch in AACAAGCTGATAAACATTTAAAGAG, even though AAAAA does not appear exactly in this string. Keep this in mind while solving the following problem.

### Frequent Words with Mismatches Problem

Find the most frequent k-mers with mismatches in a string.

    Given: 

A string Text as well as integers k and d.

    Return: 

All most frequent k-mers with up to d mismatches in Text.

Sample Dataset

    ACGTTGCATGTCGCATGATGCATGAGAGCT
    4 1

Sample Output

    GATG ATGC ATGT

In [58]:
from itertools import product
import sys
sys.path.append("../")
import common

In [59]:
def get_frequent_words(dna, k, d):
    kmers = {''.join(kmer):0 for kmer in product('ACGT', repeat = k)} 
    max_count = 0
    for i in range(len(dna) - k + 1):
        for kmer in kmers.keys():
            if common.get_hamming_distance(dna[i:i+k], kmer) <= d:
                kmers[kmer] += 1
                if max_count < kmers[kmer]:
                    max_count = kmers[kmer]
    
    return ' '.join([key for (key, value) in kmers.items() if value == max_count])

In [60]:
def get_faster_frequent_words(dna, k, d):
    freq_patterns = set()
    freq_array = [0] * 4**k
    close = [0] * 4**k
    for i in range(len(dna) - k + 1):
        neighborhood = common.get_neighbors(dna[i:i+k], d)
        for pattern in neighborhood:
            index = common.pattern_to_number(pattern)
            close[index] = 1
    for i in range(4**k):
        if close[i] == 1:
            pattern = common.number_to_pattern(i, k)
            freq_array[i] = common.get_approximate_pattern_count(dna, pattern, d)
    max_count = max(freq_array)
    for i in range(4**k):
        if freq_array[i] == max_count:
            pattern = common.number_to_pattern(i, k)
            freq_patterns.add(pattern)
    return ' '.join(freq_patterns)

In [61]:
def get_faster_frequent_words_by_sorting(dna, k, d):
    freq_patterns = set()
    neighborhood = []
    
    for i in range(0, len(dna) - k + 1):
        neighborhood.append(common.get_immediate_neighbors(dna[i:i+k], d))
    
    neighborhood_array = [item for sublist in neighborhood for item in sublist]
    
    index = sorted([common.pattern_to_number(pattern) for pattern in neighborhood_array])
    count = [1] * (len(neighborhood_array))

    for i in range(len(neighborhood_array) - 1):
        if index[i] == index[i+1]:
            count[i+1] = count[i] + 1
    
    max_count = max(count)

    for i in range(len(neighborhood_array)):
        if count[i] == max_count:
            pattern = common.number_to_pattern(index[i], k)
            freq_patterns.add(pattern)
    
    return ' '.join(freq_patterns)

In [62]:
# file = "rosalind_ba1i.txt" 
file = "input.txt" 
with open(file, 'r') as f:
    lines = f.readlines()
    dna  = lines[0].strip()
    k, d = map(int, lines[1].split())

print(get_frequent_words(dna, k, d))
print(get_faster_frequent_words(dna, k, d)) # more efficient approach on real data
print(get_faster_frequent_words_by_sorting(dna, k, d))

ATGC ATGT GATG
ATGT ATGC GATG
ATGT ATGC GATG
